In [ ]:
import json
import pickle
import pandas as pd

In [ ]:
train_data = []
with open('./data/train.json', encoding='utf-8') as file:
    for line in file:
        train_data.append(json.loads(line))
file.close()


dev_data = []
with open('./data/dev.json', encoding='utf-8') as file:
    for line in file:
        dev_data.append(json.loads(line))
file.close()

test_data = []
with open('./data/test.json', encoding='utf-8') as file:
    for line in file:
        test_data.append(json.loads(line))
file.close()

In [ ]:
# submit_data = []
# with open('./data/cluener_predict.json', encoding='utf-8') as file:
#     for line in file:
#         submit_data.append(json.loads(line))
# file.close()

In [ ]:
print(len(train_data))
print(len(dev_data))
print(len(test_data))

In [ ]:
# count_ = {}
# for k in range(len(train_data)):
#     ll = train_data[k]['label']
#     for k, v in ll.items():
#         for r, s in v.items():
#             if k not in count_:
#                 count_[k] = 0
#             for ss in s:
#                 count_[k] += 1

In [ ]:
def data_generator(data, Type='train'):
    '''
    Type: "train", "dev", "test"
    '''
    res = []
    for idx in range(len(data)):
        text = list(data[idx]['text'])
        if Type in ['train', 'dev']: 
            label = data[idx]['label']
            ner = ['O'] * len(text)
            for k, v in label.items():
                for j, r in v.items():
                    for l in r:
                        for n, m in enumerate(range(l[0], l[1]+1)):
                            if n == 0:
                                ner[m] = 'B-' + k  
                            else:
                                ner[m] = 'I-' + k
            if Type == 'dev':
                res.append([str(idx), ' '.join(text), ' '.join(ner)])
            else:
                res.append([' '.join(text), ' '.join(ner)])
        elif Type == 'test':
            res.append([str(data[idx]['id']), ' '.join(text)])
    return res

In [ ]:
train = data_generator(train_data, 'train')
dev = data_generator(dev_data, 'dev')
test = data_generator(test_data, 'test')

In [ ]:
train = pd.DataFrame(train)
dev = pd.DataFrame(dev)
test = pd.DataFrame(test)

train.columns = ['content', 'ner']
dev.columns = ['id', 'content', 'ner']
test.columns = ['id', 'content']

In [ ]:
from sklearn.model_selection import train_test_split
train, valid = train_test_split(train, train_size=0.9, random_state=42)
train = train.reset_index(drop=True)
valid = valid.reset_index(drop=True)

In [ ]:
# train.to_csv('./data/cluener_train_v2.csv', index=False)
# valid.to_csv('./data/cluener_valid_v2.csv', index=False)

In [ ]:
dev.to_csv('./data/cluener_dev_v2.csv')

In [ ]:
test.to_csv('./data/cluener_test_v2.csv')

In [ ]:
tags = []
for k in data['ner']:
    for j in k.split(' '):
        tags.append(j)
pd.Series(tags).value_counts()[1:]/pd.Series(tags).value_counts()[1:].sum() * 100

In [ ]:
tags = set()
for k in data['ner']:
    for j in k.split(' '):
        tags.update([j])

In [ ]:
ner2int = dict(zip(tags, range(len(tags))))

In [ ]:
# with open('./token_info/ner2int.pickle', 'wb') as output:
#     pickle.dump(ner2int, output)
# output.close()